In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

!pip install --upgrade huggingface_hub transformers
!pip install --upgrade requests
!pip install --upgrade tqdm
!pip install --upgrade decorator
!pip install --upgrade typing-extensions
!pip install --upgrade torch
!pip install torch==2.5.1
!pip install fastai==2.7.18
!pip install torchaudio==2.5.1
!pip install torchvision==0.20.1+cu121
!pip uninstall sympy -y
!pip install sympy==1.13.1



/kaggle/input/plasmopara-halstedii/all_secreted.fasta
/kaggle/input/plasmopara-halstedii/all_non_secreted.fasta
/kaggle/input/non-secreted/all_non_secreted_phal.fasta
/kaggle/input/non-secreted/all_non_secreted_pinf.fasta
/kaggle/input/non-secreted/all_non_secreted_psoj.fasta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 87.8 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.29.0
    Uninstalling huggingface-hub-0.29.0:
      Successfully uninstalled huggingface-hub-0.29.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.2
    Uninstalling decorator-4.4.2:
      Successfully uninstalled decorator-4.4.2
ERROR: pip's depend

In [2]:
import os
import subprocess
import logging
import sympy
from joblib import load

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define paths
pooe_repo = "https://github.com/zzdlabzm/POOE"
pooe_dir = "/kaggle/working/POOE"
model_script = os.path.join(pooe_dir, "scripts", "model.py")
model_path = os.path.join(pooe_dir, "model", "SVM_model_1.pkl")
output_pkl = "/kaggle/working/POOE/tmp_dir/prottrans_emb.pkl"
tmp_dir = "/kaggle/working/POOE/tmp_dir"
specificity = "0.8"

# Ensure POOE is cloned only once
if not os.path.exists(pooe_dir):
    logging.info("Cloning POOE repository...")
    subprocess.run(f"git clone {pooe_repo} {pooe_dir}", shell=True, check=True)
else:
    logging.info("POOE repository already exists. Skipping cloning.")

# Ensure the model file exists
if not os.path.exists(model_path):
    logging.error(f" Model file not found: {model_path}. Check the repo or download it manually.")
    raise FileNotFoundError(f"Model file does not exist: {model_path}")

# Install dependencies if not already installed
dependencies = [
    "huggingface_hub", "transformers", "requests", "tqdm", "decorator",
    "typing-extensions", "torch", "fastai", "torchaudio", "torchvision", "sympy", "biopython"
]

logging.info("Installing required dependencies...")
subprocess.run("pip install --upgrade " + " ".join(dependencies), shell=True, check=True)

# Fix sympy version issue
logging.info("Fixing sympy version...")
subprocess.run("pip uninstall sympy -y && pip install sympy==1.13.1", shell=True, check=True)

# Verify sympy installation
import sympy
logging.info(f"Sympy version: {sympy.__version__}")


In [8]:


generate_script = "/kaggle/working/POOE/features/ProtTrans/generate_prottrans.py"
model_script= "/kaggle/working/POOE/scripts/model.py"
input_fasta = "/kaggle/input/not-secreted/all_non_secreted_pult.fasta" #input fasta file each for secreted and non-secreted proteins
output_pkl = "/kaggle/working/POOE/features/ProtTrans/not_secreted.npy" #embedding output generated 
tmp_dir = "/kaggle/working/POOE/tmp_dir"

cmd = f"python {generate_script} {input_fasta} {output_pkl} {tmp_dir}"

try:
    result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("Embedding generation successful!")
    print(result.stdout.decode())  # Print the standard output
except subprocess.CalledProcessError as e:
    print(f"Error while running generate_prottrans.py: {e}")
    print(e.stderr.decode())  # Print the standard error output



Embedding generation successful!
Using cuda:0
Read 14337 sequences.
Example:
PYU1_T012888
MATAMNSYIMLTPTSSNAAANTNTNNSSNNSNNSNKVASAPASASSDLVARKWAKRKLNMELKEHVRAQRNSAAVASHKKLHGSDDDDLEPTDTDKSSSDVDDDDQQHDDADMDDSMVLGEFPTYVCELHDPKLILIAEHIQYNRIDIAPFDPPTSSSSSSSPSSAKDAKRKKTRYASSDAIFQYELSVGPTQEIFECTPAKVLELYKVLSKDAKIAEQLRFIETNYFRDGLLGSLKRLYTSPAKLSDEMIEVLEAISDVDAVLTHSAYLELIGATKGSSVYREIANLVQQHAQTPRSHPSSTCMCDAYHKSVDRFGAFYKEQAASAGNPERFSTVVSESLMWSRGVLSFYTDTVYMLDKREFRPTRGLKKCLIYGFGRRKLFEIARTSRKEWSIRHYHYGELYTLSLERYLHGMKTRHMVAIKRMVPDLQHGGLRQENVCFVKKSKKTHGYRCLLMGEVISRGKVTTSISIEHPKSPEGDPHYHYVTMSEISGAARESVCSSTLSVPADYHKHLQRLHVGGGSDVVAFLALAASYDILVGAITYWMEKKY

############# EMBEDDING STATS #############
Total number of per-residue embeddings: 0
Total number of per-protein embeddings: 14337
Time for generating embeddings: 100.3[m] (0.420[s/protein])

############# END #############
----------
PYU1_T000960 ['PYU1_T000960__0_1000']
PYU1_T012837 ['PYU1_T012837__0_1000', 'PYU1_T012837__1_1000', 'PYU1_T012837__2_1000

In [25]:


# Define the correct model directory
model_dir = "/kaggle/working/POOE/model/"

# Load all models with absolute paths
model1 = load(os.path.join(model_dir, "SVM_model_1.pkl"))
model2 = load(os.path.join(model_dir, "SVM_model_2.pkl"))
model3 = load(os.path.join(model_dir, "SVM_model_3.pkl"))
model4 = load(os.path.join(model_dir, "SVM_model_4.pkl"))
model5 = load(os.path.join(model_dir, "SVM_model_5.pkl"))

print("All models loaded successfully!")
model_dir = "/kaggle/working/POOE/model/"


✅ All models loaded successfully!
✅ All model files are present!
❌ Error while running model.py: Command 'python /kaggle/working/POOE/scripts/model.py /kaggle/working/POOE/features/ProtTrans/combined_orthologs.npy /kaggle/working/POOE/tmp_dir 0.8' returned non-zero exit status 1.
Traceback (most recent call last):
  File "/kaggle/working/POOE/scripts/model.py", line 21, in <module>
    model1 = load('../model/SVM_model_1.pkl')
  File "/usr/local/lib/python3.10/dist-packages/joblib/numpy_pickle.py", line 650, in load
    with open(filename, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: '../model/SVM_model_1.pkl'



In [26]:
#for different specificiteies
import os
import subprocess

# Set paths
pooe_dir = "/kaggle/working/POOE"
input_fasta = "/kaggle/working/input.fasta"
tmp_dir = os.path.join(pooe_dir, "tmp_dir")
os.makedirs(tmp_dir, exist_ok=True)

# Define output embedding file
output_pkl = os.path.join(tmp_dir, "prottrans_emb.pkl")

# Step 1: Generate embeddings once
generate_script = os.path.join(pooe_dir, "features", "ProtTrans", "generate_prottrans.py")
cmd1 = f"python {generate_script} {input_fasta} {output_pkl} {tmp_dir}"
print(f"🧠 Generating embeddings...\n{cmd1}")
subprocess.run(cmd1, shell=True, check=True)

# Step 2: Run model.py across different specificities
model_script = os.path.join(pooe_dir, "scripts", "model.py")
specificities = ["0.5", "0.6", "0.7", "0.8", "0.9"]

for spec in specificities:
    print(f"🔍 Running model with specificity {spec}...")
    cmd2 = f"python {model_script} {output_pkl} {tmp_dir} {spec}"
    subprocess.run(cmd2, shell=True, check=True)

print("✅ All specificities completed! Check tmp_dir for multiple jobresult_*.txt files.")


🧠 Generating embeddings...
python /kaggle/working/POOE/features/ProtTrans/generate_prottrans.py /kaggle/working/input.fasta /kaggle/working/POOE/tmp_dir/prottrans_emb.pkl /kaggle/working/POOE/tmp_dir
🔍 Running model with specificity 0.5...
🔍 Running model with specificity 0.6...
🔍 Running model with specificity 0.7...
🔍 Running model with specificity 0.8...
🔍 Running model with specificity 0.9...
✅ All specificities completed! Check tmp_dir for multiple jobresult_*.txt files.
